# Frequency Dependent Selection
![frequency dependent selection](../resources/freq_dep_selection.png)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib widget

## Vellend's frequency-dependent selection

Vellend again argues that he is talking about the same thing as Chesson in this section. It is not obvious to me why the comparisons he makes are true but it is also not obvious to me why they shouldn't be. I'm hopeful that he gets into it in later chapters.

Vellend implements frequency dependent selection using two parameters and this system is sort of conceptually a full replacement of the math surrounding constant selection. Instead of an unchanging `fit_ratio` ($k_1/k_2$) we care about its average and a dependence on frequency ($-\rho$). I've called it $-\rho$ because its pretty clearly not "niche overlap" which many authors call $\rho$. I'm going to use slightly different names in the code but their meaning is the same.

| Vellend's R       | Pat's python     | Chesson's Concept   | Math symbol       |
| :---------------- | :--------------- | :------------------ | -----------------:|
| `fit.ratio.avg`   | `avg_fit_ratio`  | fitness differences | $\left \langle \frac{k_1}{k_2}\right \rangle$ |
| `freq_dep`        | `freq_dep`       | niche differences   | $-\rho$            |

As far as I can tell, the phrases attributed to Chesson are _not_ terms that Chesson actually uses instead calling $k_1/k_2$ "fitness ratio" (sometimes, oh chesson) and $\rho$ niche overlap. In fact, I'm not entirely sure niche differences is supossed to correspond to $-\rho$ but thats my best guess. I should also note that the more recent Chesson (and many others') works use $f$ for fitness instead of $k$ but we are reserving $f$ for species _frequencies_ rather than _fitnesses_.

Vellend claims that this correspondance means that his description of frequency-dependent selection captures the "crux of 'modern coexistence theory'".

Anyway, the idea here is to define some monotonic function of frequency that describes fitness. 

Naturally, the simplest case is a line but we need it to be symmetrical about `avg_fit_ratio` which defines the fitness advantage for species 1 when $f_1 = 0.5$. For that reason we will use a linear function for the logarithm of fitness.
$$
    \log\frac{k_1}{k_2} = -\rho(f_1 - 0.5) + \log\left \langle \frac{k_1}{k_2}\right \rangle
$$

We can solve this explicitly for $k_1/k_2$ by taking the exponential of both sides. This gives us:
$$
    \frac{k_1}{k_2} = \left \langle \frac{k_1}{k_2}\right \rangle e^{-\rho(f_1 - 0.5)}

$$
So that doesn't look like a line

...but its _basically_ a line. Let's take a look.

## Oscillations from the delayed Negative-frequency dependence

In [3]:
# set up plot and define a range of frequencies
fig, ax = plt.subplots()
f1 = np.arange(0, 1, 0.01)

# put some values in for the previously described parameters
avg_fit_ratio = 1.1
freq_dep = [-0.2, -0.4, -0.6]

# first we'll put in a dashed grey line for the neutral case
ax.axhline(1, linestyle='--', color='grey', alpha=0.4)

# we'll also put in a dotted line for the fitness when sspecies are equally frequent
ax.axvline(0.5, linestyle='-.', color='grey', alpha=0.4)

# plot some lines for each frequency dependence
for fd in freq_dep:
    ax.plot(f1, avg_fit_ratio * np.exp(fd*(f1 - 0.5)), label=r'$-\rho=${}'.format(fd))

# axes labels etc.
ax.set_xlabel('Frequency of species 1')
ax.set_ylabel('Fitness ratio 1:2')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[(&#39;Home&#39;, &#39;Reset original view&#39;, &#39;home&#39;, &#39;home&#39;), (&#39;Back&#39;, &#39;Back to previous …

## Oscilations
Vellend also offers the fact that the math allows for pretty extreme oscillations due to species overshooting the balanced fitness frequencies. We implement this by only updating the current `fit_ratio` once per year, rather than once every timesetp. I've hard coded the extreme `freq_dep` value used in his figures which along with the delay can be turned on and off with the `delay` parameter.

Not quite a line but monotonic and pretty dang close.

## Implementation of Vellend's model

In [7]:
def moran_freq_dep_selection(n_indiv, n_years, init_f1, avg_fit_ratio, freq_dep, delay):
    """ Implements the Moran model including constant selection and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        init_f1: float
            initial frequency of species 1
        avg_fit_ratio: float
            fitness ratio of species 1 and 2 when both are in equal frequency
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # set up an empty array for the simulated frequencies
    # initialize with the given frequency. python counts from 0, sorrryyyy
    moran = np.zeros((n_years, 2))
    moran[0] = np.array([init_f1, 1 - init_f1])

    # get a vector representing the community. vellend calls this COM
    # this just makes a random vector drawn from the initial frequency
    comm = np.random.choice([0, 1], size=n_indiv, replace=True, p=moran[0])

    # Vellend says that freq_deq < -10 must be true to notice oscillations so if the box is ticked we'll do that
    if delay:
        freq_dep = -10

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in range(1, n_years):

        # if we're delaying the frequency dependence we do that here for each year
        if delay:
            f1 = np.sum(comm == 0) / n_indiv
            fit_ratio = avg_fit_ratio * np.exp(freq_dep*(f1 - 0.5))

        # for each year we potentially replace each individual so we can loop
        # over that as well
        for indiv in range(n_indiv):
            # get probability of species 1
            f1 = np.sum(comm == 0) / n_indiv

            # calculate the fitness ratio and probability of species 1 reproduction
            if not delay:
                fit_ratio = avg_fit_ratio * np.exp(freq_dep*(f1 - 0.5))
            pr1 = fit_ratio * f1 / (fit_ratio * f1 + (1 - f1))

            # replace one individual with another
            death = np.random.randint(n_indiv)
            birth = np.random.choice([0, 1], p=[pr1, 1 - pr1])
            comm[death] = birth
        
        # when we're done looping over all of the individuals we can update
        # frequencies for the year (the timescale we care about)
        f1 = np.sum(comm == 0) / n_indiv
        moran[year] = np.array([f1, 1 - f1])

    return moran

In [11]:
def draw_simulation(iterations=10, individuals=250, years=50, initial_freq=0.5, avg_fit_ratio=1.0, freq_dep=0.0, delay=False):
    # the plot bit, this just makes a blank figure
    fig, ax = plt.subplots(ncols=3, figsize=(10,4), gridspec_kw = {'width_ratios':[2, 3, 1]})
    # plot the frequency dependence as in Ch. 6
    # frequency range and neutral line as above
    f1 = np.arange(0, 1, 0.01)
    ax[0].axhline(1, linestyle='--', color='grey', alpha=0.4)
    # dependence curve
    ax[0].plot(f1, avg_fit_ratio * np.exp(freq_dep*(f1 - 0.5)))
    ax[0].set_xlabel('Frequency of species 1')
    ax[0].set_ylabel('Fitness ratio 1:2')

    # trajectory labels
    ax[1].set_xlabel('Years')
    ax[1].set_ylabel('Species 1 frequency')
    ax[1].set_ylim([0, 1])
    # distribution labels and bins
    hist_bins = np.arange(0, 1.1, 0.1)
    ax[2].set_xlabel('Count')
    ax[2].set_xlim([0, iterations])
    ax[2].set_ylim([0, 1])

    # we're going to plot a distribution too need an array for it
    dist = np.zeros(iterations)

    # we run the simulation and draw a trajectory
    for i in tqdm(range(iterations)):
        simulation = moran_freq_dep_selection(individuals, years, initial_freq, avg_fit_ratio, freq_dep, delay)
        ax[1].plot(range(years), simulation[:, 0], c='C0')
        # add final freq to our dist array
        dist[i] = simulation[-1, 0]

    # plot the distribution too
    ax[2].hist(dist, bins=hist_bins, orientation='horizontal')
    ax[2].axhline(np.mean(dist), linestyle='--')


    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, iterations=(5, 50, 5), individuals=(10, 1000, 10), years=(5, 200, 5),
                  initial_freq=(0.0, 1.0, 0.01), avg_fit_ratio=(0.67, 1.5, 0.01), freq_dep=(-1, 1, 0.01), delay=(False))

interactive(children=(IntSlider(value=10, description=&#39;iterations&#39;, max=50, min=5, step=5), IntSlider(value=25…

&lt;function __main__.draw_simulation(iterations=10, individuals=250, years=50, initial_freq=0.5, avg_fit_ratio=1.0, freq_dep=0.0, delay=False)&gt;